In [ ]:
import pandas as pd
import re
#import unidecode
import numpy as np
import matplotlib.pyplot as plt
import datetime
from google.colab import drive
from tqdm.notebook import tqdm
tqdm.pandas()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Seasons in data
seasons = [2021,2020,2019,2018,2017,2016,2015]
#How many time steps to include in input
day_look_back = 30
injury_path = '/content/drive/MyDrive/data_490/injury_data'

In [ ]:
#Creates a list of injury dfs where where each entry in list is a season injury df
injury_dfs = []
for i in range(len(seasons)):
    injury_dfs.append(pd.read_csv(f'{injury_path}/injury_{seasons[i]}.csv'))
    #reneames column containing player name to player_name
    injury_dfs[i] = injury_dfs[i].rename(columns={injury_dfs[i].columns[0]:'player_name','Dates':f'Dates_{seasons[i]}', 'Injury':f'injury_{seasons[i]}'})
    #changes column name of column containing injury dates to Dates_season
    injury_dfs[i] = injury_dfs[i][['player_name',f'Dates_{seasons[i]}',f'injury_{seasons[i]}']]
    injury_dfs[i].dropna(subset=['player_name',f'Dates_{seasons[i]}'],inplace=True)
    #formats player name
    injury_dfs[i]['player_name'] = injury_dfs[i].apply(lambda x: str(x.player_name).split(",")[0],axis=1)

In [ ]:
injury_dfs[0]

,player_name,Dates_2021,injury_2021
0,Sean Guenther,Array \n10/3 - 10/3,Back
1,Clayton Kershaw,Array \n10/2 - 10/3\n7/7 - 9/13,Arm
2,Josh Rogers,Array \n10/2 - 10/3,Hamstring
3,Joe Smith,Array \n10/2 - 10/3\n6/9 - 7/7,Undisclosed/Elbow
4,John Gant,Array \n10/2 - 10/3\n9/14 - 9/25,Groin/Abdominal
...,...,...,...
484,Oliver Drake,Array \n4/1 - 10/3,Arm
485,Jace Fry,Array \n4/1 - 6/26,Back
486,Jose Soriano,Array \n4/1 - 10/3,Elbow Tommy John
487,Edwar Colina,Array \n4/1 - 10/3,Elbow


In [ ]:
#merges all injurys df into one injury df with each year column containing all injuries of that year
injury_df = injury_dfs[0].merge(injury_dfs[1],on=['player_name'], how='outer')
for i in range(2,len(injury_dfs),1):
    injury_df = injury_df.merge(injury_dfs[i],on=['player_name'], how='outer')
injury_df

,player_name,Dates_2021,injury_2021,Dates_2020,injury_2020,Dates_2019,injury_2019,Dates_2018,injury_2018,Dates_2017,injury_2017,Dates_2016,injury_2016,Dates_2015,injury_2015
0,Sean Guenther,Array \n10/3 - 10/3,Back,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Clayton Kershaw,Array \n10/2 - 10/3\n7/7 - 9/13,Arm,Array \n7/23 - 8/2,Back,Array \n3/28 - 4/15,Shoulder,Array \n6/1 - 6/23\n5/6 - 5/31,Back/Biceps,Array \n7/24 - 9/1,Back,Array \n6/27 - 9/9,Back,NaN,NaN
2,Josh Rogers,Array \n10/2 - 10/3,Hamstring,NaN,NaN,Array \n6/26 - 9/29,Elbow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Joe Smith,Array \n10/2 - 10/3\n6/9 - 7/7,Undisclosed/Elbow,NaN,NaN,Array \n3/28 - 7/12,Knee ACL,Array \n6/10 - 7/3,Elbow,Array \n6/19 - 7/22,Shoulder,Array \n8/17 - 9/1\n6/5 - 7/1,Hamstring,NaN,NaN
4,John Gant,Array \n10/2 - 10/3\n9/14 - 9/25,Groin/Abdominal,Array \n9/25 - 9/27,Groin,NaN,NaN,NaN,NaN,Array \n4/2 - 5/16,Groin,Array \n6/28 - 8/21,Oblique,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,Kyuji Fujikawa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Array \n4/1 - 5/14,Groin
1335,Vic Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Array \n3/27 - 6/7,Shoulder
1336,Erik Cordier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Array \n3/27 - 5/17,Arm
1337,Tsuyoshi Wada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Array \n3/27 - 5/18,Groin


In [ ]:
#formats injury dates as yyyy-mm-dd
def get_injury_dates(df):
    injured_dates = []
    injury_types = []
    for season in seasons:
      #Adds all dates of when player goes on IL to list injured_dates
      year_injury_dates = str(df[f'Dates_{season}']).split("\n")[1:]
      year_injury_types = str(df[f'injury_{season}']).split("/")
      for i, injury in enumerate(year_injury_dates):
          injury = injury.split(" ")[0]
          injury = f'{season}-{injury.replace("/","-")}'
          if injury[-2] == "-":
              injury = injury[:-1] + injury[-1:]
          if len(year_injury_types) > i:
            injured_dates.append(injury)
            injury_types.append(year_injury_types[i])
          else:
            injured_dates.append(injury)
            injury_types.append(year_injury_types[-1])
    for i in range(1,len(injured_dates)+1):
      df[f'injury_date_{i}'] = injured_dates[-i]
      df[f'injury_{i}'] = injury_types[-i]
    return df

In [ ]:
#creates columns for each injury for each player
injury_df = injury_df.apply(get_injury_dates, axis=1)

In [ ]:
#Drops useless columns for injury_df
injury_df = injury_df.drop(columns=['Dates_' + str(season) for season in seasons]).drop(columns=['injury_' + str(season) for season in seasons])

In [ ]:
injury_df

,injury_1,injury_10,injury_2,injury_3,injury_4,injury_5,injury_6,injury_7,injury_8,injury_9,...,injury_date_10,injury_date_2,injury_date_3,injury_date_4,injury_date_5,injury_date_6,injury_date_7,injury_date_8,injury_date_9,player_name
0,Back,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Guenther
1,Back,NaN,Back,Biceps,Back,Shoulder,Back,Arm,Arm,NaN,...,NaN,2017-7-24,2018-5-6,2018-6-1,2019-3-28,2020-7-23,2021-7-7,2021-10-2,NaN,Clayton Kershaw
2,Elbow,NaN,Hamstring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2021-10-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Josh Rogers
3,Hamstring,NaN,Hamstring,Shoulder,Elbow,Knee ACL,Elbow,Undisclosed,NaN,NaN,...,NaN,2016-8-17,2017-6-19,2018-6-10,2019-3-28,2021-6-9,2021-10-2,NaN,NaN,Joe Smith
4,Oblique,NaN,Groin,Groin,Abdominal,Groin,NaN,NaN,NaN,NaN,...,NaN,2017-4-2,2020-9-25,2021-9-14,2021-10-2,NaN,NaN,NaN,NaN,John Gant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,Groin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kyuji Fujikawa
1335,Shoulder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vic Black
1336,Arm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Erik Cordier
1337,Groin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tsuyoshi Wada


In [ ]:
#Used to itterate through injury columns
num_injury_columns = len(injury_df.columns) // 2

In [ ]:
#unique injury holds all the unique injury types for each injury occurence
unique_injury = np.empty(0)
for i in range(1,num_injury_columns+1):
  unique_injury = np.append(unique_injury, injury_df[f'injury_{i}'].unique())

In [ ]:
#Drops nan values
unique_injury = unique_injury[~pd.isna(unique_injury)]

In [ ]:
#All unique injury types
unique_injury = np.unique(unique_injury)

In [ ]:
#Data frame with columns for each type of injury and player name
injury_history = pd.DataFrame(columns=unique_injury)
injury_history['player_name'] = []

In [ ]:
#row is passed from injury_df
#Each column is a type of injury and the date when that injury occured for each player
def get_injury_history(row):
  for i in range(1,num_injury_columns+1):
    injury = row[f'injury_{i}']
    injury_date = row[f'injury_date_{i}']
    if (injury != injury) or (injury_date != injury_date):
      continue
    else:
      injury_history.loc[get_injury_history.i, injury] = injury_date
      injury_history.loc[get_injury_history.i, 'player_name'] = row.player_name
  get_injury_history.i += 1
  return

In [ ]:
get_injury_history.i = 0
injury_df.apply(get_injury_history,axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1334    None
1335    None
1336    None
1337    None
1338    None
Length: 1339, dtype: object

In [ ]:
for column in injury_history.iloc[:,:-1]:
  injury_history[column] = pd.to_datetime(injury_history[column])

In [ ]:
injury_history

,Abdominal,Achilles,Ankle,Appendix,Arm,Back,Biceps,Buttocks,Calf,Chest,...,Stomach,Teeth,Thigh,Throat,Thumb,Toe,Ulnar Nerve,Undisclosed,Wrist,player_name
0,NaT,NaT,NaT,NaT,NaT,2021-10-03,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Sean Guenther
1,NaT,NaT,NaT,NaT,2021-10-02,2020-07-23,2018-05-06,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Clayton Kershaw
2,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Josh Rogers
3,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2021-10-02,NaT,Joe Smith
4,2021-09-14,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,John Gant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Kyuji Fujikawa
1335,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Vic Black
1336,NaT,NaT,NaT,NaT,2015-03-27,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Erik Cordier
1337,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Tsuyoshi Wada


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_490/processed_data/pandas_30_day')
df.game_date = pd.to_datetime(df.game_date)
df = df[['player_name', 'game_date']]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (87,93,94,95,97,103,104,105) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#Creates a data frame where each row is a player appereance with player name game date and players injury history
injury_history = df.merge(injury_history, how='left', on='player_name')

In [ ]:
#Each injury column holds the amount of time between game_date and injury_date
for column in injury_history.iloc[:,2:].columns:
  injury_history[column] = injury_history.game_date - injury_history[column]

In [ ]:
#Converts data from date time to number of days as intergers
for column in injury_history.iloc[:,2:].columns:
  injury_history[column] = (injury_history[column] - injury_history[column].min()) / datetime.timedelta(days=1)

In [ ]:
#Replaces injury dates that happened after or on game date with nan
for column in injury_history.iloc[:,2:].columns:
  injury_history[column] = injury_history[column].where(injury_history[column] >= 0, np.nan)

In [ ]:
injury_history

,player_name,game_date,Abdominal,Achilles,Ankle,Appendix,Arm,Back,Biceps,Buttocks,...,Shoulder,Stomach,Teeth,Thigh,Throat,Thumb,Toe,Ulnar Nerve,Undisclosed,Wrist
0,A.J. Achter,2015-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Achter,2015-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A.J. Achter,2015-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A.J. Achter,2015-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.J. Achter,2015-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132252,Zach Duke,2016-05-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132253,Zack Godley,2016-09-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132254,Erick Aybar,2017-04-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132255,Steven Wright,2018-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3137.0,NaN,NaN,NaN,NaN,NaN,1913.0,NaN,NaN,NaN


In [ ]:
injury_history.to_csv('/content/drive/MyDrive/data_490/injury_data/injury_history', index=False)